In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
from skimage.feature import greycomatrix, greycoprops
import os

In [ ]:
# Path to your CASIA2 dataset
PATH = '../data/CASIA2'

# Directories for authentic and tampered images
authentic_dir = os.path.join(PATH, 'Au')
tampered_dir = os.path.join(PATH, 'Tp2')

IMG_SIZE = (160, 160)

def get_file_list_and_labels(directory, label):
    file_list = []
    labels = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_list.append(os.path.join(root, file))
                labels.append(label)
    return file_list, labels

# Get file lists and labels
authentic_files, authentic_labels = get_file_list_and_labels(authentic_dir, 0)
tampered_files, tampered_labels = get_file_list_and_labels(tampered_dir, 1)

# Combine authentic and tampered data
all_files = tampered_files + authentic_files
all_labels = tampered_labels + authentic_labels

In [ ]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Check if the image is loaded correctly
    if image is None:
        raise ValueError(f"Image at {image_path} could not be loaded.")
    # Resize image
    image = cv2.resize(image, IMG_SIZE)
    return image

# Cargar imágenes
images = [load_and_preprocess_image(path) for path in all_files] # leer del dataset

In [ ]:
def fourier_transform(image):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    return magnitude_spectrum

def noise_features(image):
    # modelo de ruido básico
    mean_noise = np.mean(image)
    std_noise = np.std(image)
    return mean_noise, std_noise

def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

def texture_features(image):
    g = greycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = greycoprops(g, 'contrast')
    return np.mean(contrast)

# Compatible with grey scale 
def segment_image(image, k=4):
    # Flatten the image to a 1D array suitable for k-means
    Z = image.reshape((-1, 1))

    # Convert to float32
    Z = np.float32(Z)

    # Criteria and k-means application
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret, label, center = cv2.kmeans(Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert back to uint8 and map centers to the original image
    center = np.uint8(center)
    res = center[label.flatten()]
    segmented_image = res.reshape((image.shape))

    return segmented_image

# def extract_features(image):
#     ft = fourier_transform(image)
#     nf = noise_features(image)
#     ed = edge_detection(image)
#     tf = texture_features(image)
#     seg = segment_image(image) 
#     return np.hstack([ft.ravel(), nf[0].ravel(), nf[1].ravel(), ed.ravel(), tf.ravel(), seg.ravel()])

def extract_features(image):
    ft = fourier_transform(image).ravel()  # Aplana el resultado de la transformada de Fourier
    nf = noise_features(image)  # Retorna dos escalares
    ed = edge_detection(image).ravel()  # Aplana los bordes detectados
    tf = np.array([texture_features(image)])  # Envuelve el escalar en un arreglo
    seg = segment_image(image).ravel()  # Aplana la imagen segmentada
    
    # Concatena todas las características en un solo arreglo 1D
    return np.hstack([ft, nf[0], nf[1], ed, tf, seg])

def prepare_dataset(images):
    features = []
    labels = []  # llenar con 0 (no manipulado) y 1 (manipulado)

    total = len(images)
    count = 0

    for image, label in images:
        feat = extract_features(image)
        if len(feat) == 294915:
            features.append(feat)
            labels.append(label)
        
        count += 1
        print(f"{count}/{total}")

    return np.array(features), np.array(labels)


import os
au_paths = [os.path.join('D:\dataset--ml\CASIA2-Train-Min\Au', entry) for entry in os.listdir('D:\dataset--ml\CASIA2-Train-Min\Au')]
tp_paths = [os.path.join('D:\dataset--ml\CASIA2-Train-Min\Tp', entry) for entry in os.listdir('D:\dataset--ml\CASIA2-Train-Min\Tp')]

# Cargar imágenes
images = [(load_image(path), 0) for path in au_paths] # leer del dataset
images.extend([(load_image(path), 1) for path in tp_paths])

# Preparar conjunto de datos
X, y = prepare_dataset(images)


In [2]:
print("X_train shape:", X.shape)
print("y_train shape:", y.shape)

X_train shape: (577, 294915)
y_train shape: (577,)


In [3]:
model = svm.SVC()
model.fit(X, y)

SVC()

In [ ]:
import os
au_test_paths = [os.path.join('D:\dataset--ml\CASIA2-Test\Au', entry) for entry in os.listdir('D:\dataset--ml\CASIA2-Test\Au') if entry.endswith(".jpg")]
tp_test_paths = [os.path.join('D:\dataset--ml\CASIA2-Test\Tp', entry) for entry in os.listdir('D:\dataset--ml\CASIA2-Test\Tp') if entry.endswith(".jpg")]

# Cargar imágenes
images = [(load_image(path), 0) for path in au_test_paths[:100]] # leer del dataset
images.extend([(load_image(path), 1) for path in tp_test_paths[:100]])

# Preparar conjunto de datos
X_test, y_test = prepare_dataset(images)

In [5]:
# Evaluar modelo
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7450980392156863
